In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 

In [ ]:
# データ読み込みと前処理
import pandas as pd

data = pd.read_excel('data/settlement/general/DL202377001.xlsx')
columns = data.columns
data = data.drop(axis=1, labels=columns[1:5])

# 府省データの抽出
ministry_index = data.dropna(subset=columns[0]).drop(axis=1, labels=columns[5:9]).iloc[:, 0].index
ministry = data.dropna(subset=columns[0]).drop(axis=1, labels=columns[5:9])

# 府省ごとの円グラフ作成関数
def plot_ministry_pie(ministry_name, values, labels, total):
    plt.figure(figsize=(8, 8))
    
    # パーセントと絶対値を表示するカスタム関数
    def autopct_format(pct):
        amount = int(pct/100. * total)
        return f'{pct:.1f}%\n({amount:,})'
    
    # 円グラフ描画
    wedges, texts, autotexts = plt.pie(
        values,
        labels=labels,
        autopct=autopct_format,
        startangle=90,
        textprops={'fontsize': 10}
    )
    
    # タイトルに府省名と総額を表示
    plt.title(
        f'{ministry_name}\nTotal Budget: ¥{total:,}', 
        fontsize=14,
        pad=20
    )
    
    # 凡例を追加
    # plt.legend(
    #     wedges,
    #     [f'{l} - ¥{v:,}' for l, v in zip(labels, values)],
    #     title="Budget Items",
    #     loc="center left",
    #     bbox_to_anchor=(1, 0, 0.5, 1)
    # )
    
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    plt.close()

# 府省ごとの主要予算項目を分析
ministry_data = {}
for i in range(len(ministry)):
    ministry_name = ministry.iloc[i, 0]
    
    # 府省データの抽出
    if i < len(ministry)-1:
        ministry_data[ministry_name] = data.iloc[ministry_index[i]:ministry_index[i+1]-1]
    else:
        ministry_data[ministry_name] = data.iloc[ministry_index[i]:len(data)-1]
    
    # 予算項目データの処理
    item_data = ministry_data[ministry_name].dropna(subset=columns[8]).dropna(axis=1)
    item_data = item_data.sort_values(by=columns[9], ascending=False)
    
    # 上位10項目のみ表示（多すぎると見づらいため）
    top_items = item_data.head(10)
    
    # 総額計算
    total_budget = int(top_items.iloc[:, 1].sum())
    
    # 円グラフ描画
    plot_ministry_pie(
        ministry_name=ministry_name,
        values=top_items.iloc[:, 1],
        labels=top_items.iloc[:, 0],
        total=total_budget
    )
    
    # データ確認用に出力
    print(f"\n=== {ministry_name} ===")
    print(f"Total Budget: ¥{total_budget:,}")
    print(top_items)